In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import pickle
import datetime as dt

#Useful packages for building deep neural networks. 
import tensorflow as tf 
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D,Flatten,Dense,Dropout, Reshape,MaxPooling2D,Conv2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense

#Additional library which we will use for preprocessing our image data before training our model and to provide some specific evaluation metrics.
import sklearn
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report

In [ ]:
#---------------- PREPARE DATAFRAME FOR MODEL TRAINING ----------------------------
X_train, y_train = np.array(train_df.loc[:, train_df.columns != 'target']),np.array(train_df.loc[:,'target'])
print('training features shape:{}'.format(X_train.shape))
print('training target shape:{}\n'.format(y_train.shape))


#Apply SMOTE oversampling and random undersampling to handle the class imbalance
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
# oversample = SMOTE(sampling_strategy=0.1) #Generate new examples
# undersample = RandomUnderSampler(sampling_strategy=0.15) #undersample until we get a more equal distribution
# X_train,y_train = oversample.fit_resample(X_train, y_train)
# X_train,y_train = undersample.fit_resample(X_train, y_train)

print(X_train.shape,y_train.shape)


# #Since we have a class imbalance let's create a dictionary with class weights to balance this. This step helps the model give equal attention to less frequent training examples, be making mistakes
#on these examples more costly.
classes = np.unique(y_train,return_counts=True)[0]
class_weights_arr = sklearn.utils.class_weight.compute_class_weight(class_weight = 'balanced', classes = classes, y = y_train)

class_weights_dict = {} #input to model.fit requires dictionary
for i in classes:
    class_weights_dict[i] = class_weights_arr[i]
print("target attribute weights to handle class imbalance:{}".format(class_weights_dict))

In [ ]:

#----------------- TRAIN & EVALUATE MODEL ------------------
#Set parameters
lr = 0.0001
momentum = 0.9
batch_size = 32

# from keras import Sequential
model = Sequential()
# model.add(layers.Dense(256, activation="relu", input_dim=X_train.shape[1]))
model.add(Dense(128,activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(64,activation="relu"))
model.add(Dense(1,activation="sigmoid"))

model.compile(loss='binary_crossentropy', #integer encoded target labels, so sparse 
              optimizer='adam',
              metrics=['accuracy',tf.keras.metrics.AUC()])

model.fit(X_train,
          y_train,
          validation_split=0.1,
          batch_size = batch_size, 
          class_weight = class_weights_dict,
          epochs=10
          )

In [ ]:
#----------- DENSE NEURAL NETWORK -----------

In [ ]:
#Set parameters
lr = 0.0001
momentum = 0.9
batch_size = 64
# class_weights_dict_alt = {0:0.25,1:32} #Manuel weights to see if we can increase predictive performance.

# from keras import Sequential
model = Sequential()
model.add(layers.Dense(256, activation="relu", input_dim=X_train.shape[1]))
model.add(Dropout(0.3))
model.add(Dense(128,activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(32,activation="relu"))
model.add(Dense(1,activation="sigmoid"))

model.compile(loss='binary_crossentropy', #integer encoded target labels, so sparse 
              optimizer='adam',
              metrics=['accuracy',tf.keras.metrics.AUC()])

model.fit(X_train,
          y_train,
          validation_split=0.1,
          batch_size = batch_size, 
          class_weight = class_weights_dict,
          epochs=5
          )

# predictions_proba = model.predict(X_test)
# predictions = []
# for x in predictions_proba:
#   if x>0.5:
#     predictions.append(1)
#   else:
#     predictions.append(0)

# #AUC Curve is desirable here to evaluate the effect of different cut-off values for the predictions

# print('Overall classification report:') 
# print(classification_report(y_test,predictions))

# print('\nConfusion matrix:')
# sklearn.metrics.ConfusionMatrixDisplay.from_predictions(y_test, predictions, normalize = 'true') 
# plt.show() 

In [ ]:
#------------ HYPERPARAMETER TUNING -------------


In [ ]:
#HYPERPARBAND ALGORITHM
# SOURCE: https://towardsdatascience.com/hyperparameter-tuning-with-kerastuner-and-tensorflow-c4a4d690b31a
# !pip install keras-tuner #When running for the first time again
import keras_tuner as kt
import tensorflow_addons as tfa

#get base input shape
print(X_train.shape)
in_shp = list(X_train.shape[1:]) #47 dimensions
print(in_shp)
#input_nn = tensorflow.keras.Input(shape = (1,400)) #2D input for Convolutional layers
epochs = 20

def build_model(hp):
    """
    Builds model and sets up hyperparameter space to search. (For small dense network)
    
    Parameters
    ----------
    hp : HyperParameter object
        Configures hyperparameters to tune.
        
    Returns
    -------
    model : keras model
        Compiled model with hyperparameters to tune.
    """
    # Initialize sequential API and start building model.
    model = tf.keras.Sequential()
    model.add(tf.keras.Input(shape=in_shp))
    # Tune the number of hidden layers and units in each.
    # Number of hidden layers: 1 - 5
    # Number of Units: 32 - 256 with stepsize of 32
    for i in range(1, hp.Int("num_layers", 2, 6)):
        model.add(
            tf.keras.layers.Dense(
                units=hp.Int("units_" + str(i), min_value=32, max_value=256, step=32),
                activation="relu")
            )
        
        # Tune dropout layer with values from 0 - 0.3 with stepsize of 0.1.
        model.add(tf.keras.layers.Dropout(hp.Float("dropout_" + str(i), 0, 0.3, step=0.1)))
    
    # Add output layer.
    model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
    
    # Tune learning rate for Adam optimizer with values from 0.01, 0.001, or 0.0001
    hp_learning_rate = hp.Choice("learning_rate", values=[1e-2, 1e-3, 1e-4])
    
    # Define optimizer, loss, and metrics
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=hp_learning_rate),
                  loss=tf.keras.losses.BinaryCrossentropy(),
                  metrics=["accuracy",tfa.metrics.F1Score()])
    
    return model

# Instantiate the tuner. HyperBand algorithm used for its effectiveness

#Factor and max_epochs determine how many random models our search starts off to consider (log3(20+1)) = 3. So every iteration
#Considers 3 models in this case. It recognizes the base search space by permutations of the hp() objects defined in build_model
tuner = kt.Hyperband(build_model,
                     kt.Objective("val_f1", direction="max"), #documentation on flexible objective setting: https://keras.io/guides/keras_tuner/getting_started/
                     max_epochs=epochs,
                     factor=3, 
                     hyperband_iterations=10)
                     #directory="kt_dir", #directory to save progress to
                     #project_name="kt_hyperband") #projectname = subdirectory under the main directory


# Display search space summary
tuner.search_space_summary()

#tuner.search method takes similar input to mode.l.fit()
stop_early = tf.keras.callbacks.EarlyStopping(monitor=tfa.metrics.F1Score(), patience=5)
tuner.search(X_train, y_train, epochs=epochs, validation_split=0.2, callbacks=[stop_early], verbose=2)